**Dataset(intents) for Chatbot**

In [2]:
%%writefile intents.json
{
    "intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }]
}

Writing intents.json


In [20]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [28]:
with open("intents.json", "r") as file:
  data = json.load(file)

In [29]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])
  if intent['tag'] not in labels:
    labels.append(intent['tag'])

num_classes = len(labels)
print(num_classes)

8


In [30]:
lbls = training_labels.copy()

In [31]:
label_encoder = LabelEncoder()
lbls = label_encoder.fit_transform(lbls)

In [12]:
labelencoder = LabelEncoder()
training_labels = to_categorical(labelencoder.fit_transform(training_labels))

In [13]:
training_labels

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 

In [15]:
vocab_size = 1000
embedding_dim = 16
oov_tok = "<OOV>"
max_len = 20

In [17]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(training_sequences, truncating='post', maxlen=max_len)

In [21]:
model = Sequential()

# Input Layer
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())

# 1 - Hidden Layer
model.add(Dense(200, activation="relu"))
model.add(Dropout(0.2))

# 2 - Hidden Layer
model.add(Dense(200, activation="relu"))
model.add(Dropout(0.2))

# Output Layer
model.add(Dense(num_classes, activation="softmax"))

In [22]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 200)               3400      
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 1

In [24]:
from tensorflow.keras.callbacks import ModelCheckpoint

num_epochs = 500
checkpointer = ModelCheckpoint(filepath="/content/chatbot.hdf5", verbose=1, save_best_only=True)

In [25]:
history = model.fit(padded_sequences, np.array(training_labels), epochs=num_epochs, callbacks=[checkpointer], verbose=1)

Epoch 1/500
2/2 [==============================] - 3s 22ms/step - loss: 2.0804 - accuracy: 0.1212
Epoch 2/500
2/2 [==============================] - 0s 12ms/step - loss: 2.0778 - accuracy: 0.1515
Epoch 3/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0776 - accuracy: 0.2424
Epoch 4/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0747 - accuracy: 0.1515
Epoch 5/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0744 - accuracy: 0.1515
Epoch 6/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0748 - accuracy: 0.1818
Epoch 7/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0742 - accuracy: 0.1212
Epoch 8/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0763 - accuracy: 0.1515
Epoch 9/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0736 - accuracy: 0.1818
Epoch 10/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0723 - accuracy: 0.2121
Epoch 11/500
2/2 [=======

In [26]:
model.save("chatbot.hdf5")

In [32]:
import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(label_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
!pip install colorama

In [36]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('/content/chatbot.hdf5')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hi
ChatBot: Hello
User: what
ChatBot: Have a nice day
User: quit
